Задание:
1.	Загрузить файл длиной не менее 2000 символов. 
2.	Составить программу, которая считает число уникальных слов в тексте (без критерия схожести)
3.	Составить программу, которая считает число гласных и согласных букв. 
4.	Составить программу, которая считает число предложений, их длину и число (количество) раз использования каждого слова в тексте (с критерием схожести, критерий схожести слов выбрать самостоятельно, например, spacy (en_core_web_sm) или расстояние Левенштейна). 
5.	Вывести 10 наиболее часто встречаемых слов. 

***
В данном блокноте выполняем пункт 4 и 5 задания.

***
Анализ по данным пунктам задания:
a) Для подсчета числа предложений можно находить знаки конца предложения (точку, вопросительный и восклицательный знак). Если между предыдущим таким знаком (или началом текста) и текущим знаком находится хотя бы один алфавитно-цифровой знак, то это и будет предложение.
Таким образом, можно разбить текст на список предложений. Длина списка будет числом предложений в тексте.
b) Для отыскания длины каждого предложения можно предварительно каждое предложение в списке от знаков препинания, заменив их пробелами, и расщепить на список слов с помощью соответствующей процедуры Python. Длина этого списка будет длиной предложения.
c) Для нахождения числа раз использования каждого слова в тексте с использованием критерия схожести используем библиотеку spacy. 
Вначале мы уберем из текста все знаки препинания и разделим текст на отдельные слова. Получим множество всех слов текста. Используя процедуру, отработанную при выполнении второго пункта задания, уберем незначащие слова из списка.
Далее для каждого слова из множества, используя функции библиотеки spacy, найдем список похожих на него слов. Далее, имея набор списков похожих слов, для каждого слова каждого списка найдем количество его вхождений в исходный очищенный текст. Взяв сумму числа вхождений для слов каждого списка, получим искомое число раз использования каждого слова в тексте.
Чтобы не было путаницы с регистрами текста, предварительно приводим весь текст к нижнему регистру.

Тогда план работы по пункту 2 следующий.

1. Загрузить текст из файла.
2. Перевести текст в нижний регистр.
3. Проходить строку, включающую исследуемый текст, знак за знаком, находить знак конца предложения, то есть точку, восклицательный или положительный знак, и его положение в тексте. Для точки проверять, чтобы справа и слева не было цифр, иначе это будет десятичная точка числа, а не знак конца предложения.
4. Взяв слайс между предыдущим и текущим знаком конца предложения, проверить, есть ли в этом слайсе алфавитно-цифровые знаки. В случае наличия считаем полученный слайс очередным предложением текста и заносим его в список предложений.
5. Далее брать по одному предложению из списка и создавать из него новую строку, включающую только алфавитно-цифровые знаки, остальные знаки заменять пробелами. Затем расщеплять полученную строку процедурой split и находить длину полученного списка. Это будет длина предложения в словах. Выводить результат на экран.
6. Убрать в тексте, полученном в пункте 2, знаки препинания и шире - все неалфавитные знаки, заменив их пробелами.
7. Разбить текст на слова процедурой split, считая пробелы разделителями.
8. Из полученного списка слов убрать все стоп-слова.
9. Получить множество уникальных (пока без критерия схожести) слов текста, преобразовав список в множество.
10. Загрузить модель spacy en_core_web_sm. Для каждого слова множества с помощью данной модели найти схожие с ним слова. Список похожих слов добавить в словарь в качестве ключа. Эти похожие слова удалить из множества уникальных слов. Повторять для каждого следующего слова, пока слова в множестве не кончатся.
11. Для каждого ключа полученного словаря проверить число вхождений в очищенный текст каждого слова ключа. Суммарное число вхождений присвоить как значения поля value данного ключа.
12. Отсортировать полученный словарь по значению ключа (дающего частотность соответствующего слова)
13. Отобрать 10 наиболее часто встречающихся слов в полученном словаре.
14. Если в полученном словаре есть еще слова с той же частотностью, что и у десятого слова, также включить их в выводимый результат.

1. Загружаем текстовый файл:

In [50]:
import string, spacy

file_name = 'GospelJohn.txt' # file with text
file_name = 'example.txt' # file with text
try:
    f = open(file_name,"r") # open file for reading
    text = f.read()         # reading file 
    f.close()               # closing file
    print('Read successfully')
except:
    print('Error reading file!')

Read successfully


2. Переводим текст в нижний регистр.

In [51]:
# to lower case:
text = text.lower()

3. Проходить строку, включающую исследуемый текст, знак за знаком, находить знак конца предложения, то есть точку, восклицательный или положительный знак, и его положение в тексте. Для точки проверять, чтобы справа и слева не было цифр, иначе это будет десятичная точка числа, а не знак конца предложения.
4. Взяв слайс между предыдущим и текущим знаком конца предложения, проверить, есть ли в этом слайсе алфавитно-цифровые знаки. В случае наличия считаем полученный слайс очередным предложением текста и заносим его в список предложений.

In [ ]:
# "left" - variable for left-hand end of a sentence
right = -1 # variable for right-hand end of a sentence
i = -1 # counter
sentences_length_list = list() # list for sentences lengths
for c in text:
    i += 1
    if c in ".?!":
        if c == '.' and text[i-1].isnumeric and text[i+1].isnumeric:
            print(f'The dot is reached, i = {i}')
            print(text[i-1])
            print(text[i+1])
            continue
        # new boundaries of a string:
        left = right + 1
        right = i
        astring = text[left:right] # extract the string
        print(f'String:\n{astring}')
        # if the string contains alphanumeric symbols, we suppose for it to be a sentence
        if len([ch for ch in astring if ch.isalnum()]) > 0:
            sentence_alphanumeric = "".join([ch if ch.isalnum else " " for ch in astring])
            sentence_length = len(sentence_alphanumeric.split()) # length of sentence in words
            sentences_length_list.append(sentence_length)
print(f'Lengths of sentences in the text (in words): {sentences_length_list}')

The dot is reached, i = 3
u


The dot is reached, i = 17
s
 
The dot is reached, i = 35
t
a
String:
ou.
the thing is. the other is not.are you here
String:
jackie
Lengths of sentences in the text (in words): [10, 1]


In [48]:
str = 'The string'
str[0:3]

'The'

In [16]:
a = "the string 123!?<>"
lst = [ch for ch in a if ch.isalnum()]
print(lst)

['t', 'h', 'e', 's', 't', 'r', 'i', 'n', 'g', '1', '2', '3']


Убираем не-алфавитные знаки и разбиваем текст на слова, помещаем их в список.

In [17]:
# replace non-alphabetic signs by whitespaces:
text_no_signs = "".join([c if c.isalpha() else " " for c in text])

word_list = text_no_signs.split() # split text into words and put to the list

Убираем из списка лишние слова ("стоп-слова").
Для этого вначале создаем список стоп-слов
(источник: https://github.com/explosion/spaCy/blob/master/spacy/lang/en/stop_words.py)
Также мы немного расширяем этот список для учета устаревших слов, фигурирующих в нашем тексте

In [18]:
STOP_WORDS = set(
    """
a about above across after afterwards again against all almost alone along
already also although always am among amongst amount an and another any anyhow
anyone anything anyway anywhere are around as at

back be became because become becomes becoming been before beforehand behind
being below beside besides between beyond both bottom but by

call can cannot ca could

did do does doing done down due during

each eight either eleven else elsewhere empty enough even ever every
everyone everything everywhere except

few fifteen fifty first five for former formerly forty four from front full
further

get give go

had has have he hence her here hereafter hereby herein hereupon hers herself
him himself his how however hundred

i if in indeed into is it its itself

keep

last latter latterly least less

just

made make many may me meanwhile might mine more moreover most mostly move much
must my myself

name namely neither never nevertheless next nine no nobody none noone nor not
nothing now nowhere

of off often on once one only onto or other others otherwise our ours ourselves
out over own

part per perhaps please put

quite

rather re really regarding

same say see seem seemed seeming seems serious several she should show side
since six sixty so some somehow someone something sometime sometimes somewhere
still such

take ten than that the their them themselves then thence there thereafter
thereby therefore therein thereupon these they third this those though three
through throughout thru thus to together too top toward towards twelve twenty
two

under until up unless upon us used using

various very very via was we well were what whatever when whence whenever where
whereafter whereas whereby wherein whereupon wherever whether which while
whither who whoever whole whom whose why will with within without would

yet you your yours yourself yourselves
""".split()
)

contractions = ["n't", "'d", "'ll", "'m", "'re", "'s", "'ve"]
STOP_WORDS.update(contractions)

for apostrophe in ["‘", "’"]:
    for stopword in contractions:
        STOP_WORDS.add(stopword.replace("'", apostrophe))

obsoletisms = ["art", "hath", "thou", "thee", "unto"] # obsolete word forms
STOP_WORDS.update(obsoletisms)

Убираем из списка слова, если они входят в список стоп-слов:

In [19]:
word_list_new = list()
for aword in word_list:
    if aword not in STOP_WORDS:
        word_list_new.append(aword)

# print(len(word_list))
# print(len(word_list_new))
word_list = word_list_new


Проходим по всему набору слов и вносим ранее не встреченные слова в словарь, для ранее встреченных увеличиваем value на единицу

In [20]:
word_dict = dict() # empty dictionary for all words
# putting all the words into dict:
for aword in word_list:
    if aword in word_dict: # if already in dict
        word_dict[aword] += 1            
    else:                  # if not in dict yet
        word_dict[aword] = 1  
                 
# print(word_dict)

Находим число уникальных слов (без критерия похожести; "стоп-слова" уже удалены и не учитываются)

In [21]:
key_number = len(word_dict.keys())
print(f'The number of the unique words in the text: {key_number}')

The number of the unique words in the text: 208


Сортируем словарь по увеличению частоты встречаемости слова, извлекаем десять последних (то есть самых частых) слов.
Если 11-е, 12-е и т.д. слова встречаются так же часто, как и десятое, выводим также и их, потому что по факту они все делят десятое место в списке наиболее частых слов

In [22]:
# sorting the list of the most frequent words:
word_dict_sorted = sorted(word_dict.items(), key = lambda item: item[1])
print('The most frequent words:')

# extracting the most frequent words
for i in range(10):
    aword = word_dict_sorted[-i-1][0]
    frequency = word_dict_sorted[-i-1][1]
    print(f'{i+1}. The word "{aword}" has the frequency {frequency}')

# extracting also the 11th, 12th etc, if the frequency is the same as for the 10th one:
while word_dict_sorted[-i-2][1] == word_dict_sorted[-10][1]:
    aword = word_dict_sorted[-i-2][0]
    frequency = word_dict_sorted[-i-2][1]
    print(f'{i}. The word "{aword}" has frequency {frequency}')
    i += 1

The most frequent words:
1. The word "And" has the frequency 20
2. The word "I" has the frequency 16
3. The word "saith" has the frequency 13
4. The word "said" has the frequency 12
5. The word "Jesus" has the frequency 12
6. The word "God" has the frequency 12
7. The word "John" has the frequency 9
8. The word "He" has the frequency 8
9. The word "saw" has the frequency 7
10. The word "sent" has the frequency 6
9. The word "man" has frequency 6


Итак, нами был проделан анализ англоязычного текста в отношении количества уникальных слов без объединения словоформ по критерию схожести.

Выводы:
1. При анализе числа уникальных слов в тексте следует удалять т. наз. "стоп-слова", то есть артикли, предлоги и прочие слова, не несущие смысловой нагрузки.
2. Для исключения ситуации, когда слова в начале и середине предложения считаются за разные слова за счет отличия регистра первой буквы, следует привести слова перед анализом к одному регистру.
3. В классическом англоязычном тексте 17 в. на ~1000 слов (4102 знака без учета пробелов) после удаления стоп-слов оказалось 170 уникальных слов (без дальнейшего объединения словоформ по критерию схожести), то есть число уникальных значащих слов примерно в пять раз меньше, чем общее число всех слов.
4. Десять отобранных уникальных слов с максимальной частотностью дают хорошее представление о тематике текста.